<div align="center">

# <span style="color: #3498db;">CA2 - Genetic and Games Algorithm</span>

**<span style="color:rgb(247, 169, 0);">[Minoo Sanjari]</span> - <span style="color:rgb(143, 95, 195);">[810102560]</span>**

</div>

# <span style="color: #3498db;">Genetic Algorithm</span>

## Imports

In [107]:
%matplotlib inline

In [108]:
from bisect import bisect_left
from dataclasses import dataclass
from itertools import accumulate
from random import randrange, random, uniform, sample
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm

In [109]:
class ImageAnalysis:
    def __init__(self, image: Image, rows: int, columns: int):
        self._image = image
        self._rows = rows
        self._columns = columns
        self._pieces = self._split_image()
        ######## TWO OPTIONS

        ######## OPTION1
        # self._dissimilarity_matrix = self._calculate_dissimilarity_matrix()
        ########
        
        ######## OPTION2(faster)
        N = self._rows * self._columns
        self._lr_dissimilarity_matrix = np.zeros((N,N), dtype=np.float32)
        self._td_dissimilarity_matrix = np.zeros((N,N), dtype=np.float32)
        self._calculate_dissimilarity_matrices_fast()
        ########

    @property
    def pieces(self) -> dict:
        return self._pieces

    # NOT USED IN BOTH SLOW OR FAST VERSIONS
    def get_dissimilarity(self, ids: tuple, orientation: str) -> float:
        return self._dissimilarity_matrix[ids][orientation]

    def _split_image(self) -> dict:
        width, height = self._image.size
        piece_width = width // self._columns
        piece_height = height // self._rows

        pieces = {}
        for i in range(self._rows):
            for j in range(self._columns):
                left = j * piece_width
                top = i * piece_height
                right = left + piece_width
                bottom = top + piece_height
                piece = self._image.crop((left, top, right, bottom))
                pieces[i * self._columns + j] = piece
        return pieces

    def _calculate_dissimilarity_matrix(self) -> dict:
        matrix = {}
        for i in range(self._rows * self._columns):
            for j in range(self._rows * self._columns):
                if i == j:
                    continue
                matrix[i, j] = {
                    "L-R": self._dissimilarity_measure(self._pieces[i], self._pieces[j], "L-R"),
                    "T-D": self._dissimilarity_measure(self._pieces[i], self._pieces[j], "T-D"),
                }
        return matrix

    def _dissimilarity_measure(self, piece1: Image, piece2: Image, orientation: str) -> float:
        """TODO: Implement this method to calculate the dissimilarity between two pieces."""
        
        piece1_arr = np.asarray(piece1).astype(np.float32)
        piece2_arr = np.asarray(piece2).astype(np.float32)
        edge1=[]
        edge2=[]

        if orientation=="L-R" :
            edge1 = piece1_arr[:, -1, :]   # last column of pixels
            edge2 = piece2_arr[:, 0, :]    # first column of pixels
        elif orientation=="T-D" :
            edge1 = piece1_arr[-1, :, :]   # last row of pixels
            edge2 = piece2_arr[0, :, :]    # first row of pixels
        
        dissimilarity = np.sum((edge1 - edge2)**2)
        return float(dissimilarity)
    
    def _calculate_dissimilarity_matrices_fast(self):
        """
        Calculates and populates the two N x N dissimilarity matrices by
        extracting borders from the PIL Image objects in self._pieces.
        """
        N = self._rows * self._columns
        
        # Pre-convert all pieces to NumPy arrays for faster access
        piece_arrays = [np.asarray(self._pieces[k]).astype(np.float32) for k in range(N)]
        #pieces_arrays is a N*N*3 array
        for i in range(N):
            piece_i_arr = piece_arrays[i]
            
            # Right border of piece i (last column of pixels)
            right_border_i = piece_i_arr[:, -1, :] 
            # Bottom border of piece i (last row of pixels)
            bottom_border_i = piece_i_arr[-1, :, :] 

            for j in range(N):
                piece_j_arr = piece_arrays[j]

                # Left border of piece j (first column of pixels)
                left_border_j = piece_j_arr[:, 0, :]
                self._lr_dissimilarity_matrix[i, j] = self._dissimilarity_measure_fast(
                    right_border_i, 
                    left_border_j
                )

                # Top border of piece j (first row of pixels)
                top_border_j = piece_j_arr[0, :, :]
                self._td_dissimilarity_matrix[i, j] = self._dissimilarity_measure_fast(
                    bottom_border_i, 
                    top_border_j
                )

    def _dissimilarity_measure_fast(self, border_1: np.ndarray, border_2: np.ndarray) -> float:
        """
        Fast SSD calculation between two pre-extracted NumPy border arrays.
        """
        return np.sum((border_1 - border_2)**2)



## Image Analysis

### ImageAnalysis Class

The `ImageAnalysis` class is responsible for two main tasks:

1.  **Splitting the Image**: It takes the source image and splits it into a grid of smaller puzzle pieces.
2.  **Calculating Dissimilarity**: It computes a "dissimilarity matrix," which stores a score of how well any two pieces fit together. This is crucial for the genetic algorithm's fitness function.

You will need to implement the `_dissimilarity_measure` method, which is the core of the analysis. The rest of the class is provided for you.

### Task: Implement the `dissimilarity_measure` Method

The `dissimilarity_measure` method calculates how different two puzzle pieces are when placed next to each other. This is a key part of the fitness function, as it determines how well pieces fit together.

#### What You Need to Do:

-   **Implement the `_dissimilarity_measure` method** in the `ImageAnalysis` class.
-   The method should take two `PIL.Image` objects (`piece1`, `piece2`) and an `orientation` string (`"L-R"` for left-right or `"T-D"` for top-down).
-   It should return a single `float` value representing the dissimilarity.

#### Hints:

1.  **Convert Images to NumPy Arrays**: Use `np.asarray(piece)` to convert the `PIL.Image` objects into NumPy arrays. This makes it easier to perform mathematical operations on the pixel data.

2.  **Extract Edges**: Depending on the `orientation`, you need to extract the pixels from the touching edges of the two pieces.
    -   For `"L-R"` (Left-Right), you need the **right edge of `piece1`** and the **left edge of `piece2`**.
    -   For `"T-D"` (Top-Down), you need the **bottom edge of `piece1`** and the **top edge of `piece2`**.

3.  **Calculate Dissimilarity**: The dissimilarity can be calculated as the **sum of squared differences** between the pixel values of the two edges. The formula is:

    ```
    dissimilarity = sum((edge1 - edge2)^2)
    ```

    You can use `np.sum()` and basic arithmetic operations on the NumPy arrays to compute this value.

## **REPORT* *

Based on the given hints I implemented _dissimilarity_measure , choosing edge1 and edge2 based on orientation and then finding their dissimilarity with SSD(np.sum((edge1 - edge2)**2)).

## Size Detector

In [110]:
class SizeDetector:
    def __init__(self, image: Image):
        self._image = image

    def detect(self) -> tuple:
        width, height = self._image.size
        aspect_ratio = width / height

        for i in range(1, int(width**0.5) + 1):
            if width % i == 0:
                j = width // i
                if abs(j / i - aspect_ratio) < 0.1:  # Heuristic for aspect ratio
                    return (i, j) if width > height else (j, i)
        return (1, width) # Fallback

## Size Detector

### SizeDetector Class

The `SizeDetector` class automatically determines the size of the puzzle pieces from the input image. It works by analyzing the image's properties and looking for repeating patterns that suggest the dimensions of the pieces.

This class is provided to you and does not require any modifications. You can use it to get the piece size before starting the genetic algorithm.

## Individual

### Task: Implement the `Individual` Class

The `Individual` class represents a possible solution to the jigsaw puzzle. It stores the puzzle pieces and evaluates how well they fit together. You will need to implement several parts of this class to calculate the fitness and manage the puzzle pieces.

#### 1. Fitness Calculation (`fitness` property)

- **What You Need to Do**:
  - Implement the `fitness` property to calculate how well the pieces fit together.
  - The fitness value should be based on how the pieces align (i.e., their edges) both horizontally and vertically.
  - The fitness function should return an inverse fitness value based on the dissimilarities between adjacent pieces.

  **Hint**:
  - Use the `ImageAnalysis.get_dissimilarity(ids, orientation)` function to get the dissimilarity between adjacent pieces.
  - Calculate the dissimilarity between adjacent pieces both **horizontally** and **vertically** (Left-Right and Top-Down).

#### 2. Core Methods (`__init__`, `piece_size`, `piece_by_id`)

- **What You Need to Do**:
  - Implement the `__init__` method to initialize the puzzle with pieces and store the number of rows and columns.
  - Implement the `piece_size()` method to return the size of a single piece.
  - Implement the `piece_by_id()` method to retrieve a piece by its unique ID.

---

#### Parts You **Do Not Need to Implement**:

- **`edge` Method**:
  - This method checks the neighboring piece's ID based on orientation (top, right, bottom, left). It is useful for advanced puzzle assembly but is **not necessary** for this task. You can **skip** implementing this method.

- **`to_image` Method**:
  - This method reassembles the individual puzzle into a complete image. It is used for **visualization** of the solution and is **not required** for fitness evaluation. You can **skip** implementing this method.

In [111]:
class Individual:
    def __init__(self, pieces: dict, rows: int, columns: int, initial_state: List[int] = None):
        """TODO: Initialize the individual with pieces, rows, and columns."""
        self._pieces = pieces
        self._rows = rows
        self._columns = columns
        total_pieces = rows * columns

        if initial_state == None:
            self._chromosome = np.random.permutation(total_pieces).reshape(rows, columns)
        else:
            self._chromosome = np.array(initial_state).reshape(rows, columns)

    @property
    # SLOW VERSION
    # def fitness(self) -> float:
    #     """TODO: Calculate the fitness of the individual."""
    #     # HINT: Fitness is the inverse of the sum of dissimilarities.
    #     total_dissimilarity = 0.0
    #     # Loop through each cell and sum dissimilarities with right and bottom neighbors
    #     for i in range(self._rows):
    #         for j in range(self._columns):
    #             current_id = self._chromosome[i, j]

    #             # Right neighbor (L-R)
    #             if j < self._columns - 1:
    #                 right_id = self._chromosome[i, j + 1]
    #                 total_dissimilarity += self._image_analysis.get_dissimilarity((current_id, right_id), "L-R")

    #             # Bottom neighbor (T-D)
    #             if i < self._rows - 1:
    #                 bottom_id = self._chromosome[i + 1, j]
    #                 total_dissimilarity += self._image_analysis.get_dissimilarity((current_id, bottom_id), "T-D")

    #     return 1.0 / (1.0 + total_dissimilarity)

    #FAST VERSION
    def fitness(self) -> float:
        """
        Vectorized implementation using the pre-calculated N x N matrices.
        This is the change that gives the massive speed boost.
        """
        lr_matrix = self._image_analysis._lr_dissimilarity_matrix
        td_matrix = self._image_analysis._td_dissimilarity_matrix

        lr_mismatch = lr_matrix[
            self._chromosome[:, :-1].flatten(),
            self._chromosome[:, 1:].flatten()
        ].sum()
        
        td_mismatch = td_matrix[
            self._chromosome[:-1, :].flatten(),
            self._chromosome[1:, :].flatten()
        ].sum()

        return 1.0 / (lr_mismatch + td_mismatch + 1.0)

    def piece_size(self) -> tuple:
        """TODO: Return the size of a single piece."""
        # with asumption that all pices have same size
        sample_piece = next(iter(self._pieces.values()))
        # sample_piece = self._pieces[0]
        return sample_piece.size  # returns (width, height)

    def piece_by_id(self, piece_id: int) -> Image:
        """TODO: Return the piece image by its ID."""
        return self._pieces[piece_id]

    def to_image(self) -> Image:
        """Reassembles the individual into a single image."""
        piece_width, piece_height = self.piece_size()
        canvas_width = self._columns * piece_width
        canvas_height = self._rows * piece_height
        canvas = Image.new('RGB', (canvas_width, canvas_height))

        for i in range(self._rows):
            for j in range(self._columns):
                piece_id = self._chromosome[i, j]
                piece = self.piece_by_id(piece_id)
                canvas.paste(piece, (j * piece_width, i * piece_height))
        return canvas

    def __getitem__(self, key):
        return self._chromosome[key]

    def __repr__(self):
        return f"Individual(fitness={self.fitness})"
    
    def set_image_analysis(self, image_analysis):
        """Set the reference ImageAnalysis object (used to access dissimilarities)."""
        self._image_analysis = image_analysis

## **REPORT* *

The Individual class stores self._pieces which is a dict data type that holds each piece image (actual object) and its signature piece id. Data self._chromosome is a 2D array (with the row and column shape like the original image does) of pieces' ids. If the input initial_state (which is a list of pieces' ids) equals NONE then we put a random sequence of ids with the chromosome shape for self._chromosome, otherwise self._chromosome gets the initial_state. So basically this class holds the pieces of the image as the objects of image, one chromosome, which is a layout (permutation), is equal to input state (initial_state) or a local random generated sequence.

fitness function works like this: for each piece, we find the dissimilarity of the current piece with its right and bottom piece (if they exist) by giving their id and orientation to get_dissimilarity function of the ImageAnalysis class. In the ImageAnalysis class, we already computed all dissimilarities of any two pieces together and stored them in _dissimilarity_matrix and have access to any of them with get_dissimilarity function. We sum all of these values to show the total dissimilarity of this chromosome. Since the fitness has an inverse connection with dissimilarity we should return 1/total_dessimilarity. Because of the case total_dissimilarity=0 (happens when all the pieces are in their right place), I added 1 to total_dissimilarity to avoid division by 0.

For having access to get_dissimilarity, we should set an image_analysis object for this class or assume that there will be a global object of image_analysis and we have access to it in this class as well. I went with the first option and therefore I declared a set_image_analysis function just to set a local image_analysis for this class.

For the piece_size function, I assumed that all the pieces have the same size. so for getting the first piece I used

next(iter(self._pieces.values()))

and then the size of this piece represents all pieces size.

## Selection

### Task: Implement the `roulette_selection` Function

The **roulette selection** process is part of the genetic algorithm and is used to select pairs of individuals from the population based on their **fitness**. The fitter individuals are more likely to be selected for reproduction, but all individuals still have a chance of being selected, even if their fitness is low.

---

#### **Hints**:

- You can use `random.uniform(0, total_sum)` to generate a random number between 0 and the total fitness sum.
- The `bisect_left` function helps find the position in the probability intervals where the random number fits.
- The **elites** are guaranteed to move to the next generation without crossover, so they should be excluded from the random selection process.

---

### Task: Implement Another Selection Method

Besides `roulette_selection`, there are other selection strategies. Implement one of the following:

1.  **Tournament Selection**:
    -   Randomly select a few individuals (a "tournament").
    -   The individual with the best fitness in the tournament is chosen as a parent.
    -   Repeat to select the second parent.

2.  **Rank Selection**:
    -   Rank the individuals by fitness.
    -   Selection probability is based on rank, not fitness value. This can prevent premature convergence.

In [112]:
def roulette_selection(population: List[Individual], elite_size: int) -> List[Tuple[Individual, Individual]]:
    """TODO: Implement roulette wheel selection to choose pairs of parents."""
    # Exclude elite individuals
    non_elites = population[elite_size: ]
    total_fitness = sum(ind.fitness for ind in non_elites)
    cumulative_probs = []
    cumulative_sum = 0
    for ind in non_elites:
        cumulative_sum += ind.fitness / total_fitness
        cumulative_probs.append(cumulative_sum)
    
    def select_one():
        r = random()  # random float in [0,1)
        idx = bisect_left(cumulative_probs, r)
        return non_elites[min(idx, len(non_elites) - 1)]
    
    pairs = []
    for _ in range(len(non_elites)):
        parent1 = select_one()
        parent2 = select_one()
        pairs.append((parent1, parent2))
    
    return pairs

def your_other_selection_method(population: List[Individual], elite_size: int, k: int = 3) -> List[Tuple[Individual, Individual]]:
    """TODO: Implement another selection method like Tournament or Rank selection."""
    non_elites = population[elite_size: ]
    
    def select_one():
        cooshen_inds = sample(non_elites, k)
        winner = max(cooshen_inds, key=lambda ind: ind.fitness)
        return winner
    
    pairs = []
    for _ in range(len(non_elites)):
        parent1 = select_one()
        parent2 = select_one()
        pairs.append((parent1, parent2))
    
    return pairs

## **REPORT* *

### roulette_selection:

First, we exclude elite individuals (like the project said to) cause they go to the next generation without crossover. This gives us non_elites list of individuals. Instead of using random.uniform, I normalized the distribution by dividing each non_elite individual by its fitness to relate a probability to each of them. In a loop over non_elites, I added each individual's fitness to get cumulative_sum and add it to cumulative_probs list. By doing this, an individual with a higher fitness (bigger probability) gets a wider area (range) between 0 to 1, so this individual has more chance that the random number (between 0 to 1) falls in its range. I defined a local helper function, select_one, for selecting an individual as a parent. Doing this we will have two chosen parents as a pair. roulette_selection returns a list of pairs with length equal to len(non_elites).

### your_other_selection_method:

I implemented Tournament selection. Again, from non_elites list we should chose two parents as a pair. But this time with a different selection strategy using local helper function (select_one). this function randomly pickes k sample from non_elites and then choses the one with the biggest fitness.

## Crossover

### Task: Implement the Crossover Logic

Crossover is where the "genetic" part of the algorithm happens. You'll create a new "child" individual by combining the traits of two "parent" individuals. The goal is to build a better-assembled puzzle by picking the best-fitting pieces from the parents.

This crossover strategy is complex. It starts with a random piece and then iteratively adds adjacent pieces based on a priority system.

#### Crossover Priority System:

1.  **Shared Pieces (Highest Priority)**: If two parents agree on a neighboring piece, that piece is chosen. This is a strong indicator of a correct fit.
2.  **Buddy Pieces (Medium Priority)**: A "buddy piece" is a pair of pieces that are each other's best match. If one of these is a neighbor, it's a good candidate.
3.  **Best Match (Lowest Priority)**: If neither of the above is found, choose the piece that has the lowest dissimilarity score (the best fit) from the remaining pieces.

---

#### Your Tasks:

1.  **Implement Piece Selection Methods**:
    -   `_get_shared_piece()`: Find pieces that are neighbors in **both** parents.
    -   `_get_buddy_piece()`: Find "buddy" pieces (mutual best friends).
    -   `_get_best_match_piece()`: Find the piece with the best dissimilarity score among the available pieces.

2.  **Implement the `Crossover` Class**:
    -   Use a `heapq` (min-heap) to manage candidate pieces based on the priority system.
    -   Implement the `run()` method to build the child's chromosome by selecting from the candidates.
    -   You can create different versions of the `add_piece_candidate` method to experiment with different strategies (e.g., one using only shared and best-match pieces, and another using all three).

3.  **Experiment and Compare**:
    -   After implementing the `Crossover` class, you can create variations of it. For example, one version might only use "shared" and "best match" pieces, while another could use all three priorities.
    -   Compare how these different crossover strategies affect the speed and quality of the solution. Does including "buddy pieces" help? Is it worth the extra computation?


In [113]:
import heapq

SHARED_PIECE_PRIORITY = -10
BUDDY_PIECE_PRIORITY = -1

def complementary_orientation(orientation):
    return {"T": "D", "R": "L", "D": "T", "L": "R"}.get(orientation)

class Crossover:
    def __init__(self, parent1: Individual, parent2: Individual, analysis: ImageAnalysis):
        self._parents = (parent1, parent2)
        self._analysis = analysis
        self._pieces_length = len(parent1._pieces)
        self._child_rows = parent1._rows
        self._child_columns = parent1._columns
        self._kernel = {}
        self._taken_positions = set()
        self._candidate_pieces = []
        self._min_row, self._max_row, self._min_column, self._max_column = 0, 0, 0, 0

    def run(self):
        # TODO: Implement the main crossover loop.
        # This should initialize the kernel and then process candidate pieces
        # until the child chromosome is full.
        self._initialize_kernel()
        while not self._is_kernel_full() and self._candidate_pieces :
            priority, (position,piece_id), (_,_) = heapq.heappop(self._candidate_pieces)
            if self._is_valid_piece(piece_id) and position not in self._taken_positions :
                self._put_piece_to_kernel(piece_id, position)
        return self.child()


    #THIS COULD HAD ERROR BECAUSE OF p.id for p in piece SO i CHANGED IT AND ALSO THE PROBLEM OF NOT PASSING THE NEW CHROMOSOME
    # def child(self) -> Individual:
    #     # This method is provided for you. It assembles the new child individual.
    #     pieces = [None] * self._pieces_length
    #     for piece_id, (row, column) in self._kernel.items():
    #         index = (row - self._min_row) * self._child_columns + (column - self._min_column)
    #         pieces[index] = self._parents[0].piece_by_id(piece_id)
        
    #     # Fill any remaining empty spots with unused pieces
    #     used_piece_ids = {p.id for p in pieces if p is not None}
    #     all_pieces = {p.id: p for p in self._parents[0]._pieces.values()}
    #     unused_pieces = [all_pieces[pid] for pid in all_pieces if pid not in used_piece_ids]
        
    #     for i in range(self._pieces_length):
    #         if pieces[i] is None:
    #             pieces[i] = unused_pieces.pop()

    #     return Individual(dict(zip([p.id for p in pieces], pieces)), self._child_rows, self._child_columns)

    def child(self) -> Individual:        
        # Create a flat list to represent the new chromosome
        child_state_list = [None] * self._pieces_length
        
        for piece_id, (row, column) in self._kernel.items():
            # Calculate the flat index for the new chromosome
            index = (row - self._min_row) * self._child_columns + (column - self._min_column)
            if 0 <= index < self._pieces_length:
                child_state_list[index] = piece_id

        used_piece_ids = set(self._kernel.keys())
        all_piece_ids = set(self._parents[0]._pieces.keys())
        unused_ids = list(all_piece_ids - used_piece_ids)
        
        # Fill in any remaining None spots in the chromosome with unused pieces
        for i in range(self._pieces_length):
            if child_state_list[i] is None:
                if unused_ids:
                    child_state_list[i] = unused_ids.pop()
                else:
                    # This should not happen if logic is correct, but as a fallback
                    # find *any* ID not in the list (can be slow, but safe)
                    remaining_id = (all_piece_ids - set(filter(None, child_state_list))).pop()
                    child_state_list[i] = remaining_id
        
        all_pieces_dict = self._parents[0]._pieces
        # Create the new Individual, passing the new chromosome as initial_state
        return Individual(
            all_pieces_dict, 
            self._child_rows, 
            self._child_columns, 
            initial_state=child_state_list
        )


    def _initialize_kernel(self):
        # This method is provided. It selects a random starting piece.
        root_piece_id = list(self._parents[0]._pieces.keys())[randrange(self._pieces_length)]
        self._put_piece_to_kernel(root_piece_id, (0, 0))

    def _put_piece_to_kernel(self, piece_id: int, position: tuple):
        # This method is provided. It adds a piece to the solution kernel.
        self._kernel[piece_id] = position
        self._taken_positions.add(position)
        self._update_candidate_pieces(piece_id, position)

    def _update_candidate_pieces(self, piece_id: int, position: tuple):
        # This method is provided. It finds available boundaries and adds new candidates.
        for orientation, pos in self._available_boundaries(position):
            self.add_piece_candidate(piece_id, orientation, pos)

    def add_piece_candidate(self, piece_id: int, orientation: str, position: tuple):
        """
        TODO: Implement the logic to add candidate pieces to the heap.
        This is where you will use your _get_shared_piece, _get_buddy_piece,
        and _get_best_match_piece methods to decide which piece to add.
        Remember to use the priority constants!
        """
        shared_piece = self._get_shared_piece(piece_id, orientation)
        if shared_piece is not None and self._is_valid_piece(shared_piece):
            self._add_candidate_to_heap(SHARED_PIECE_PRIORITY, shared_piece, position,(piece_id,orientation))
            return
        
        buddy_piece = self._get_buddy_piece(piece_id, orientation)
        if buddy_piece is not None and self._is_valid_piece(buddy_piece) :
            self._add_candidate_to_heap(BUDDY_PIECE_PRIORITY, buddy_piece, position,(piece_id,orientation))
            return

        best_piece, dissimilarity = self._get_best_match_piece(piece_id, orientation)
        if best_piece is not None and self._is_valid_piece(best_piece):
            self._add_candidate_to_heap(dissimilarity, best_piece, position, (piece_id, orientation))


    def _get_shared_piece(self, piece_id: int, orientation: str) -> int:
        """
        TODO: Find a piece that is a neighbor to piece_id in BOTH parents.
        Return the piece_id if found, otherwise None.
        """
        def find_neighbor(parent):
            coords = np.where(parent._chromosome == piece_id)
            if coords[0].size == 0:
                return None  # Should not happen if parents are valid
            i, j = coords[0][0], coords[1][0] 

            if orientation == "R" and j < parent._columns - 1:
                return parent._chromosome[i, j + 1]
            if orientation == "L" and j > 0:
                return parent._chromosome[i, j - 1]
            if orientation == "T" and i > 0:
                return parent._chromosome[i - 1, j]
            if orientation == "D" and i < parent._rows - 1:
                return parent._chromosome[i + 1, j]
            return None

        n1 = find_neighbor(self._parents[0])
        n2 = find_neighbor(self._parents[1])
        if n1 == n2 and n1 is not None :
            return n1 
        else :
            return None

    def _get_buddy_piece(self, piece_id: int, orientation: str) -> int:
        """
        TODO: A "buddy piece" is a pair (A, B) where B is the best match for A
        and A is the best match for B.
        Check if such a piece exists for the given orientation.
        """
        best1, _ = self._get_best_match_piece(piece_id, orientation)
        if best1 is None:
            return None
        best2, _ = self._get_best_match_piece(best1, complementary_orientation(orientation))
        if best2 == piece_id:
            return best1
        return None

    # WORKS FOR SLOW VERSION
    # def _get_best_match_piece(self, piece_id: int, orientation: str) -> tuple:
        
    #     N = self._analysis._rows * self._analysis._columns

    #     # Determine which dissimilarity key to use
    #     if orientation in ("L", "R"):
    #         key = "L-R"
    #     else:
    #         key = "T-D"

    #     best_j = None
    #     min_dissimilarity = np.inf

    #     for j in range(N):

    #         if j == piece_id or j in self._kernel:
    #             continue

    #         # orientation says the orientation from piece_id to look
    #         pair = None
    #         if orientation == "R":
    #             pair = (piece_id, j)  # (current, right_neighbor)
    #         elif orientation == "L":
    #             pair = (j, piece_id)  # (left_neighbor, current)
    #         elif orientation == "D":
    #             pair = (piece_id, j)  # (current, bottom_neighbor)
    #         elif orientation == "T":
    #             pair = (j, piece_id)  # (top_neighbor, current)

    #         if pair in self._analysis._dissimilarity_matrix:
    #             dissimilarity = self._analysis._dissimilarity_matrix[pair][key]
    #             if dissimilarity < min_dissimilarity:
    #                 min_dissimilarity = dissimilarity
    #                 best_j = j

    #     if best_j is None:
    #         return None, np.inf

    #     return best_j, min_dissimilarity

    #FAST VERSION
    def _get_best_match_piece(self, piece_id: int, orientation: str) -> tuple:
        """
        Vectorized implementation to find the best matching piece based on 
        pre-calculated N x N dissimilarity matrices.
        """
        N = self._analysis._rows * self._analysis._columns
        if orientation in ("L", "R"):
            dissimilarity_matrix = self._analysis._lr_dissimilarity_matrix
        else: # T or D
            dissimilarity_matrix = self._analysis._td_dissimilarity_matrix

        if orientation in ("R", "D"):
            # When looking for a piece to the Right (R) or Down (D) of piece_id,
            # we check how the **Right/Bottom border of piece_id (i)** matches 
            # the Left/Top border of all other pieces (j). This corresponds to row 'i'.
            # SCORE = dissimilarity_matrix[piece_id, all_j]
            scores = dissimilarity_matrix[piece_id, :].copy()
        else: # L or T
            # When looking for a piece to the Left (L) or Top (T) of piece_id,
            # we check how the **Left/Top border of piece_id (j)** matches 
            # the Right/Bottom border of all other pieces (i). This corresponds to column 'j'.
            # SCORE = dissimilarity_matrix[all_i, piece_id]
            scores = dissimilarity_matrix[:, piece_id].copy()

        # mask invalid pieces
        invalid_indices = np.zeros(N, dtype=bool)
        invalid_indices[piece_id] = True
        
        for k_id in self._kernel:
            invalid_indices[k_id] = True
            
        scores[invalid_indices] = np.inf

        min_dissimilarity = np.min(scores)

        if np.isinf(min_dissimilarity):
            # No possible piece found
            return None, np.inf
        
        best_j = np.argmin(scores)
        return int(best_j), float(min_dissimilarity)

    def _add_candidate_to_heap(self, priority: int, piece_id: int, position: tuple, relative_piece: tuple):
        # This is a helper to push candidates to the heap.
        heapq.heappush(self._candidate_pieces, (priority, (position, piece_id), relative_piece))

    # The following helper methods for boundary checks are provided for you.
    def _available_boundaries(self, row_and_column):
        (row, column) = row_and_column
        boundaries = []
        if not self._is_kernel_full():
            positions = {"T": (row - 1, column), "R": (row, column + 1), "D": (row + 1, column), "L": (row, column - 1)}
            for orientation, position in positions.items():
                if position not in self._taken_positions and self._is_in_range(position):
                    self._update_kernel_boundaries(position)
                    boundaries.append((orientation, position))
        return boundaries

    def _is_kernel_full(self):
        return len(self._kernel) == self._pieces_length

    def _is_in_range(self, pos):
        return self._is_row_in_range(pos[0]) and self._is_column_in_range(pos[1])

    def _is_row_in_range(self, row):
        return abs(min(self._min_row, row)) + abs(max(self._max_row, row)) < self._child_rows

    def _is_column_in_range(self, col):
        return abs(min(self._min_column, col)) + abs(max(self._max_column, col)) < self._child_columns

    def _update_kernel_boundaries(self, pos):
        self._min_row, self._max_row = min(self._min_row, pos[0]), max(self._max_row, pos[0])
        self._min_column, self._max_column = min(self._min_column, pos[1]), max(self._max_column, pos[1])

    def _is_valid_piece(self, piece_id):
        return piece_id is not None and piece_id not in self._kernel


## **REPORT* *

### run(self) :

First we need to pick a random piece and put it in (0,0) position. We do this by calling self._initialize_kernel() which is already given to us. After that, functions call each other iteratively and procces the evolution to find candidate_pieces (using different methods with different priorities) and push them in self._candidate_pieces, which is a min heap. Next, we pop from self._candidate_pieces and put the poped piece to kernel with two conditions that it is a valid piece ( self._is_valid_piece(piece_id) ) and its position is not already taken in the kernel. We will do this proccess until the kernel is not full and there is still piece candidate in self._candidate_pieces. For next step, we call self.child() to assemble the new child (a chromosome) as a next gen individual and complete a crossover operation.

Each element in self._candidate_pieces heap has these items with the same order: the first item is priority of the piece_candidate. The min heap dicides which element to pop based on priority. next items are a tuple of (position,piece_id) and relative_piece which is a tuple of (piece_id, orientation).

### add_piece_candidate(self, piece_id: int, orientation: str, position: tuple) :

This function implements the logic to add candidate pieces to heap. We can use three methods to find candidate piece for this function's input piece. I called these three methods based on their priority like the discription said. We first check the _get_shared_piece result and add it to candidate pieces (using self._add_candidate_to_heap() ) with two conditions that the suggested piece is not None and it is a valid piece (using self._is_valid_piece() ) and return from add_piece_candidate cuase we already found our candidate piece. If the conditions are not satisfied, then we check _get_buddy_piece result and add it with the same conditions and return. Otherwise we check the third method with the least priority which is _get_best_match_piece and add its result with the same conditions.

#### For doing the asked experiment :
For comparing the effect of using the second method (_get_buddy_piece), I will commente and uncomment this part in add_piece_candidate function:

```
buddy_piece = self._get_buddy_piece(piece_id, orientation)
        if buddy_piece is not None and self._is_valid_piece(buddy_piece) :
            self._add_candidate_to_heap(BUDDY_PIECE_PRIORITY, buddy_piece, position,(piece_id,orientation))
            return
```

### _get_shared_piece(self, piece_id: int, orientation: str) -> int :

This methods finds a shared piece using parent1 and parent2. In this functions we look for the given piece_id in the parent's chromosome, get its neighbor's id (the neighbor of piece_id in the given orientation). I implemented a local helper function find_neighbor for finding this neighbor. We do this for both parents and check if they are equal which means both parents share a neighbor for piece_id and so they both agree on the same piece to be in piece_candidate which is a strong indicator of a correct chromosome. If parents do not have the same neighbor, we return None to tell we could not find a good piece with this method.

### _get_buddy_piece(self, piece_id: int, orientation: str) -> int :

This method finds the best piece match for the given piece_id (using the other method _get_best_match_piece), then finds the best piece match for the found piece and checks if these two found piece (output of _get_best_match_piece) are the same. If they are the same that means they are matualy eachothers best match. We call these two pieces buddies. If they are not the same, we return None and say we could not find any good piece candidate for piece_id with this method.

### _get_best_match_piece(self, piece_id: int, orientation: str) -> tuple :

This method finds the best match piece for given piece_id. I used self._analysis._dissimilarity_matrix to get a list of the whole dissimilarities of piece_id in the given orientations with all existing pieces. We need to find the best piece match which means the piece with the minimum dissimilarity value, But with two conditions, that the found piece is not equal to piece_id (cause we do not look for piece_id dissimilarity with itself) and the found piece should not be used (not in self._kernel). For finding this minimum using numpy array operations (to be more efficient like the discription asks to), I defined a boolean array named "mask" to apply the two mentioned conditions. I used mask to excule the pices we do not want to check. Eliminating those pieces, we get two lists (valid_dissimilarities and valid_piece_ids) that we can simply find the minimum dissimilarity among them and return the min dissimilarity value and its coresponding piece as the best match piece.

## IMPORTANT CHANGE : 
### child(self) -> Individual: 

I had to change child function because of two crutial problems it had. One problem (it would cause error!) was this line :

```
used_piece_ids = {p.id for p in pieces if p is not None}

```

it was using p.id where p was a element in pieces which was a list of pieces with type PIL Image and its indexes represents piece id but had NO ATRIBUTE .id !

The second BIG problem was that after filling the empty pieces, when we should return the new individual as the child, we should  give Individual class's constructor the pieces' ids that we made (the new chromosome as the new child) by passing it as its initial_state. But what the provided was doing, was not giving what we made to Individual's contructor and so, the constructor would put its defualt value which is None. That will make a whole new randomly generated chromosome and ignoring the crossover result each time! Therefor I changed this function to fix these problems.


## Mutation

In [114]:
from copy import deepcopy
from random import randint
def mutation(individual: Individual, mutation_rate: float = 0.01) -> Individual:
    ## TODO
    # Copy to avoid modifying the original
    mutated = deepcopy(individual)
    
    if random() < mutation_rate:
        rows, cols = mutated._chromosome.shape

        r1, c1 = randint(0, rows - 1), randint(0, cols - 1)
        r2, c2 = randint(0, rows - 1), randint(0, cols - 1)

        # Swap the piece IDs in chromosome
        mutated._chromosome[r1, c1], mutated._chromosome[r2, c2] = (
            mutated._chromosome[r2, c2],
            mutated._chromosome[r1, c1],
        )

    return mutated

## **REPORT* *

### mutation(individual: Individual, mutation_rate: float = 0.01) -> Individual :

Mutation operation happens on a child with probibility mutation_rate. So we can generate a random number in [0,1) range and if it is less than mutation_rate, mutation will happen. For doing the mutation, we only need to pick four random int number in the range of given indevidual shape and then just swap piece ids of these two position in the chromosome.

## Genetic Algorithm

### Task: Implement the `GeneticAlgorithm` Class

This class simulates the process of evolution to solve the jigsaw puzzle using a **Genetic Algorithm**.

The genetic algorithm involves selection, crossover, and mutation processes to evolve better solutions over time.

---

In [115]:
@dataclass
class GeneticAlgorithm:
    image_path: str
    pieces_size: int
    population_size: int
    generations: int
    elite_size: int
    mutation_rate: float
    selection_method: callable
    crossover_method: callable
    score_function: callable

    def run(self) -> Individual:
        """TODO: Implement the main loop of the genetic algorithm."""
        # 1. Load image and initialize analysis
        img = Image.open(self.image_path).convert("RGB")
        if self.pieces_size is None :
            rows, cols = SizeDetector(img).detect()
        else :
            rows = self.pieces_size
            cols = self.pieces_size
        
        analysis = ImageAnalysis(img, rows, cols)

        # 2. Create initial population
        population: List[Individual] = []
        for _ in range(self.population_size):
            ind = Individual(analysis.pieces, rows, cols)
            ind.set_image_analysis(analysis)
            population.append(ind)

        for gen_num in tqdm(range(self.generations), desc="GA Generations"):
            # a. Sort population by fitness (higher to lowest)
            population.sort(key=lambda ind: ind.fitness, reverse=True)

            # visualization
            best_ind = population[0]
            # Create a NEW figure for each generation
            plt.figure(figsize=(4, 4)) 
            plt.imshow(best_ind.to_image())
            plt.title(f"Gen {gen_num}/{self.generations} | Fitness: {best_ind.fitness:.10f}")
            plt.axis('off')
            plt.show() # Forces the figure to render immediately

            # Keep elites
            next_gen: List[Individual] = population[: self.elite_size]

            # b. Select parents (selection_method returns list of (p1,p2) pairs)
            parent_pairs = self.selection_method(population, self.elite_size)
            
            #################################################
            # Ensure we only produce enough children to fill population
            num_of_needed_parentpairs = self.population_size - self.elite_size
            parent_pairs = parent_pairs[ : num_of_needed_parentpairs]
            #################################################

            # c. For each parent pair, produce a child (crossover -> mutation -> add new child to next gen)
            for (p1, p2) in parent_pairs:
                # child_candidate = self.crossover_method(p1, p2, analysis).run()
                child_candidate = Crossover(p1, p2, analysis).run()
                # Set its image analysis
                child_candidate.set_image_analysis(analysis)
                # Apply mutation
                child_after_mut = mutation(child_candidate, self.mutation_rate)
                # Ensure image analysis is still set after mutation (deepcopy may have preserved it, but be safe)
                child_after_mut.set_image_analysis(analysis)

                #################################################
                # Optionally allow custom score_function, otherwise rely on child's .fitness
                if self.score_function is not None:
                    # If user provided score_function, they can choose how to evaluate and maybe attach it.
                    _ = self.score_function(child_after_mut)
                #################################################
                next_gen.append(child_after_mut)
                # Stop early if we already filled the next generation
                if len(next_gen) >= self.population_size:
                    break

            #################################################
            # If for some reason we didn't get enough children (e.g., selection returned fewer pairs),
            # fill remaining slots with random individuals (safe fallback)
            while len(next_gen) < self.population_size:
                ind = Individual(analysis.pieces, rows, cols)
                ind.set_image_analysis(analysis)
                next_gen.append(ind)
            #################################################

            # Replace population
            population = next_gen


        # Final sort and return the best individual
        population.sort(key=lambda ind: ind.fitness, reverse=True)
        best = population[0]
        # 4. Return the best individual
        return best
    

## **REPORT* *

After opening the image, the first thing we need to do is finding the best sugested number of rows and columns using already provided SizeDetector class. Then we can split the image in the found row and column numbers using ImageAnalysis class.

The second step is to make a initial population with the given size. We do this by passing None for initial_state of Individual and the class constructor will randomly generate a chromosome. Also we should call .set_image_analysis(analysis) for each new individual (needed for computing its fitness localy).



## Running the Algorithm

In [ ]:
# TODO: Set up and run the Genetic Algorithm
# You will need to instantiate the GeneticAlgorithm class with your chosen parameters
# and then call the run() method.

# Example:
ga = GeneticAlgorithm(
    image_path='images/starry.jpg',
    pieces_size=None,
    population_size=100,
    generations=10,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
best_solution = ga.run()

plt.imshow(best_solution.to_image())
plt.axis('off')
plt.show()
best_solution.to_image().save("part1_outputs/final_puzzle_starry_2crossover.png")

KeyboardInterrupt: 

## **REPORT* * 

### The first implementation codes :

These parts are commented as SLOW VERSION in the codes. I did not delete them so I can show how the algorithm would work with the given implementations (where dissimilarity_matrix in ImageAnalysis class is a dict)

#### inputs #1 :
```
ga = GeneticAlgorithm(
    image_path='images/lion.jpg',
    piece_size=None,
    population_size=100,
    generations=10,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #1 : time:15m 35.7s 
GA Generations: 100%| | 10/10 [15:15<00:00, 91.59s/it]

![image does not exist](part1_outputs/baboon_output.png)

The answer seems complete but it takes too much time to finish the algorithm.

#### inputs #2 :
```
ga = GeneticAlgorithm(
    image_path='images/lion.jpg',
    piece_size=None,
    population_size=100,
    generations=10,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #2 : time:40m 2.4s 
GA Generations: 100%| | 10/10 [38:21<00:00, 230.10s/it]

![image does not exist](part1_outputs/lion_output.png)

It takes too much time. For fixing this problem I changed the _init in ImageAnalysis class and fitness in Individual classs. The difference is when we store disimilarities in two 2D arrays, one fore L-R and one for T-D. Eith this structure we have opene hands to use np array functions (which is much faster) instead of for loops.

#### The second implementation code :

This codes are the fast versions of the first commented implementation. The core change is that we store two disimilarity_matrix, one fore L-R () and one for T-D (). In this case we can do some operations with np array functions. I changed these codes:

_calculate_dissimilarity_matrix -> _calculate_dissimilarity_matrices_fast

_dissimilarity_measure -> _dissimilarity_measure_fast

fitness #FAST VERSION

_get_best_match_piece #FAST VERION


#### inputs #3 :
```
ga = GeneticAlgorithm(
    image_path='images/baboon.jpg',
    piece_size=None,
    population_size=100,
    generations=15,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #3 : time:1m 20.7s 
GA Generations: 100%| | 15/15 [01:17<00:00, 5.19s/it]

![image does not exist](part1_outputs/final_puzzle_baboon15gen.png)

The same inputs  (and even with more generations) gave the answer in 1 min and 20 sec which is more than 14 min faster! So we better continue with our second faster implementations.

#### inputs #4 : lets see the first gens too

```
ga = GeneticAlgorithm(
    image_path='images/island.jpg',
    piece_size=None,
    population_size=100,
    generations=10,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #4 : time:37m 49.2s

![image does not exist](part1_outputs/islandgen0of10_output.png)
![image does not exist](part1_outputs/islandgen1of10_output.png)
![image does not exist](part1_outputs/islandgen3of10_output.png)
![image does not exist](part1_outputs/islandgen5of10_output.png)
![image does not exist](part1_outputs/islandgen7of10_output.png)

GA Generations: 100%| | 10/10 [36:33<00:00, 219.39s/it]

![image does not exist](part1_outputs/final_puzzle_island10gen.png)

We can see that image gets filiped in some generations but with enough generations it can flip back.

#### inputs #5 : lets see the first gens too

```
ga = GeneticAlgorithm(
    image_path='images/chessboard.jpg',
    piece_size=None,
    population_size=100,
    generations=10,
    elite_size=20,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #5 : time:7m 2.7s

![image does not exist](part1_outputs/chessboard0of10gen_output.png)
![image does not exist](part1_outputs/chessboard2of10gen_output.png)

GA Generations: 100%| | 10/10 [6:45<00:00, 45.6s/it]

![image does not exist](part1_outputs/final_puzzle_chessboard10gen.png)

The result is not even close to the original image. The reason is explained in question 2.

#### inputs #6 : lets see the first gens too

```
ga = GeneticAlgorithm(
    image_path='images/starry.jpg',
    piece_size=None,
    population_size=100,
    generations=20,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #6 : time:12m 55.8s

![image does not exist](part1_outputs/starrygen0of20_output.png)
![image does not exist](part1_outputs/starrygen1of20_output.png)
![image does not exist](part1_outputs/starrygen6of20_output.png)

GA Generations: 100%| | 20/20 [12:43<00:00, 38.18s/it]

![image does not exist](part1_outputs/final_puzzle_starry20gen.png)

This image has a lot of texture and details and therefor it will probibility be made completly under 10 generations. But even for this easily made image, we can see clearly that the increase of generation number will make the algorithem finish later as expected.

#### inputs #7 : lets see the first gens too

```
ga = GeneticAlgorithm(
    image_path='images/starry.jpg',
    piece_size=None,
    population_size=100,
    generations=20,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### output #7 : time:10m 5.0s

![image does not exist](part1_outputs/pillars0of10gen_output.png)
![image does not exist](part1_outputs/pillars1of10gen_output.png)
![image does not exist](part1_outputs/pillars5of10gen_output.png)

GA Generations: 100%| | 10/10 [09:56<00:00, 59.63s/it]

![image does not exist](part1_outputs/final_puzzle_pillars10gen.png)

It needs more generations to be completly right.

#### inputs #8 : lets see what happens when we pass a pieces_size to algorithm insted of leting itself sets a suggested pieces_size. For seeing the better differences, I use baboon image as input cause comparing to other inputs, it seems to be the fastest to get a answer.

```
ga = GeneticAlgorithm(
    image_path='images/baboon.jpg',
    pieces_size=32/64/128,
    population_size=100,
    generations=10,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=roulette_selection, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```
#### pieces_size=32
#### output pieces_size=32: time:6m 13.7s

![image does not exist](part1_outputs/baboon0of10gen_32size_output.png)
![image does not exist](part1_outputs/baboon2of10gen_32size_output.png)

![image does not exist](part1_outputs/final_puzzle_baboon10gen_32size.png)

we see that after gen 2 it got stuck and change can not be seen even untill gen 10. We would need more generations or maybe a little bit of luck!

#### pieces_size=64
#### output pieces_size=64:

![image does not exist](part1_outputs/baboon0of10gen_64size_output.png)
![image does not exist](part1_outputs/baboon2of10gen_64size_output.png)

algotithm took more than 40 min so I stopeed it, but it seemed to got stuck after gen2.
#### pieces_size=128
#### output pieces_size=128:

After 30 min it did not even give the gen 0! So I tried pieces_size=16 so we can compare eith pieces_size=32

#### pieces_size=16
#### output pieces_size=32:

![image does not exist](part1_outputs/baboon0of10gen_16size_output.png)
![image does not exist](part1_outputs/baboon1of10gen_16size_output.png)

![image does not exist](part1_outputs/final_puzzle_baboon10gen_16size.png)


#### inputs #9 : this time we will see the effect of commenting method _get_buddy_piece in crossover class (with pieces_size=None and with baboon and starry inputs)

#### output #9 :

It took 1m 4.6s and 3gens for baboon input, and 6m 29.8s and 2gens for starry input. Comparing with outputs #3 and #6(half the gens and time), it seems not using the second method of our crossover, we would not lose anything and in some casess the algorithm is even faster! This result was expected because method 2 was basically doing (calling exactly, in our codes) the method 3 for two times and checking one extra condition. That means using method 2 like that useally would not help that much! In mwthod 2 we are returning the same piece (if it is not None) that we would return in method 3 only with the difference that it is a stronger guss for chosing pieces.

#### inputs #10 : this time we change the selection function to the tournoment selection (with pieces_size=None and using 3 methods for crossover)
```
ga = GeneticAlgorithm(
    image_path='images/baboon.jpg',
    pieces_size=None,
    population_size=100,
    generations=10,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=your_other_selection_method, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```

#### output #10 : time: 0m 49.7s

![image does not exist](part1_outputs/baboon_tournamentgen0of10_output.png)
![image does not exist](part1_outputs/baboon_tournamentgen1of10_output.png)

![image does not exist](part1_outputs/final_puzzle_baboon_tournament.png)

Comparing this time with output #3 and getting the image right after 2gens, it seems that tournoment selection worked quite better than roullet selection. Lets see this selection on another image as well.


#### inputs #11 : 
```
ga = GeneticAlgorithm(
    image_path='images/starry.jpg',
    pieces_size=None,
    population_size=100,
    generations=20,
    elite_size=10,
    mutation_rate=0.01,
    selection_method=your_other_selection_method, # or your other method
    crossover_method=None,
    score_function=None # or score_b, score_c
)
```

#### output #11 : time: 9m 27.6s

![image does not exist](part1_outputs/starry_tournamentgen0of20_output.png)
![image does not exist](part1_outputs/starry_tournamentgen1of20_output.png)
![image does not exist](part1_outputs/starry_tournamentgen2of20_output.png)

![image does not exist](part1_outputs/final_puzzle_starry_tournament.png)

comparing this time to output #6 and getting the image right after 3gens, looks that tournpment selection is working better and faster.

<div style="color:rgb(235, 66, 32); font-weight: bold;">⚠️ Important Note:</div>  

Using **NumPy arrays** allows you to perform operations on vectors **more efficiently** and **faster**.

**Avoid using `for` loops** whenever possible, as vectorized operations in NumPy are **optimized for performance** and significantly reduce execution time.

## **REPORT* * 
### Answer to the questions of part 1 :


##### *question 1*
Having 48 pieces would give 48! uniqe chromosomes (since there is 48 ids and 48 places to fill) which can be rounded to 10 to the power of 61 !

##### *question 2*
This algorithm did not work correctly on the chessboard image. The main reason is that the usual fitness function that we used for other images will not work on chessboard, becuase chessboard is made of white and black squers and each squer has a high dissimilarity on its edges with a neighbor squer and our fitness functions interpret that do a low fitness value! While that is how a chessboard is, each squer is srounded by its oposite color (high dissimilarity on the edges).Using our usual fitness function, this algorithm tries to put pieces with the same color beside each other since that is how it will get a higher fitness value (which is wrong!). by passing a different fitness function to our algorithem we can fix this problem with chessboard image.

##### *question 3*
Simple crossover methods like one-point or two-point are generally ill-suited for the jigsaw puzzle problem because they destroy crucial local information and frequently produce invalid chromosomes.

The fitness of a puzzle solution depends entirely on the connections between adjacent pieces. Simple crossovers randomly select a segment of pieces from one parent and combine it with a segment from the other. This process is highly likely to break many correct boundary connections built up in the parents, resulting in a child that is much worse than both parents.

In a permutation problem, every piece id must appear exactly once. If you use standard one-point crossover on two parents, cutting and splicing results in children that contain duplicates and miss pieces. Therefore, simple operators cannot be used directly and require complex repair mechanisms (which are often slow and ineffective).

The heuristic, neighborhood-based crossover we implemented is necessary because it prioritizes preserving and inheriting high-quality local adjacencies, which is the sole factor determining a good puzzle solution.

##### *question 4*
High selection pressure, often achieved by heavily favoring the absolute best individuals, can lead to premature convergence.

Premature Convergence: This occurs when the population quickly becomes dominated by individuals that share the same suboptimal genetic material. Because the best individuals are chosen exclusively, the algorithm rapidly loses the diversity needed to explore the entire solution space. For a jigsaw puzzle, this means the algorithm might find a good local arrangement of, say, the top-left corner, and then fail to discover better connectivity solutions for the rest of the puzzle because all subsequent generations are variations of that single, flawed local optimum.

Methods like Tournament Selection help maintain diversity by mitigating this intense pressure. In Tournament Selection, a small subset ($k$) of individuals is randomly sampled from the population, and only the best among that subset is chosen as a parent.This process gives weaker individuals a chance to compete and occasionally win a tournament if their random competitors are even weaker. By allowing individuals with potentially beneficial but currently hidden genes to survive and breed, Tournament Selection slows down the rate at which the population loses genetic material.This preserved diversity prevents the population from settling too quickly on the first good solution it finds, allowing the search process to continue exploring more varied configurations and eventually discover the fully solved puzzle.

##### *question 5*
The primary role of mutation is to restore and maintain genetic diversity by introducing entirely new genetic information that may have been lost during selection and crossover, preventing the GA from getting stuck in a local optimum.

It ensures that every possible piece placement can eventually be reached, helping the algorithm jump out of fitness valleys.

##### Risk of too low mutation rate (e.g.,0.001) (Under-exploration):

The risk is stagnation and premature convergence (similar to high selection pressure).The crossover and selection process will quickly remove genetic material that appears suboptimal, and the mutation operator won't introduce new material fast enough to replenish the lost diversity. The algorithm will get stuck in the nearest local optimum and stop improving.

##### Risk of too high mutation rate (e.g., $0.5$ or more) (Random Search):

The algorithm essentially converts into an inefficient random search.Every time a highly fit child is created through crossover, the high mutation rate immediately scrambles its hard-won, correct adjacencies. The genetic information passed down from parents is destroyed almost instantly. The algorithm spends most of its time evaluating near-random individuals, losing the power of evolutionary search and requiring excessive generations to find the solution.

# <span style="color: #3498db;">Dots and Boxes with Minimax Algorithm</span>

In this section, you will implement a **Minimax algorithm with Alpha-Beta pruning** to play the Dots and Boxes game. The game is already implemented, and you only need to focus on the AI agent that uses the Minimax algorithm to make intelligent decisions.

## Imports

In [117]:
import math
import numpy as np

from tkinter import *
from dataclasses import dataclass
from typing import List, Optional, Tuple

### **Dots and Boxes** is a classic two-player game where:
- Players take turns connecting adjacent dots with horizontal or vertical lines
- When a player completes the fourth side of a box, they score a point and get another turn
- The game ends when all lines are drawn, and the player with the most boxes wins

### Your Task

You will implement the **MinimaxAgent** class that uses:
1. **Minimax Algorithm**: A decision-making algorithm that explores the game tree to find the optimal move
2. **Alpha-Beta Pruning**: An optimization technique that reduces the number of nodes evaluated in the game tree
3. **Evaluation Function**: A heuristic to estimate the value of non-terminal game states

**Important**: You **MUST** implement Alpha-Beta pruning. This is a required optimization that significantly improves the algorithm's performance.

## GameState Class

The `GameState` class represents the current state of the game. It is **provided for you** and includes:
- `board_status`: Tracks how many edges each box has (values from 0-4)
- `row_status`: Tracks which horizontal edges are drawn
- `col_status`: Tracks which vertical edges are drawn
- `player1_turn`: Boolean indicating whose turn it is

### Key Methods (Already Implemented):
- `clone()`: Creates a copy of the game state
- `is_gameover()`: Checks if the game has ended
- `get_scores()`: Returns the current scores for both players
- `available_moves()`: Returns list of all valid moves
- `apply_move()`: Applies a move and returns whether a box was completed

In [135]:
@dataclass
class GameState:
    board_status: np.ndarray  # shape (n-1, n-1)
    row_status: np.ndarray    # shape (n, n-1)
    col_status: np.ndarray    # shape (n-1, n)
    player1_turn: bool

    def clone(self) -> "GameState":
        """Creates a deep copy of the current game state."""
        return GameState(
            board_status=self.board_status.copy(),
            row_status=self.row_status.copy(),
            col_status=self.col_status.copy(),
            player1_turn=self.player1_turn,
        )

    def get_dimensions(self) -> int:
        """Returns the number of dots (n) in the game grid."""
        return self.row_status.shape[0]

    def is_gameover(self) -> bool:
        """Checks if all edges have been drawn."""
        return (self.row_status == 1).all() and (self.col_status == 1).all()

    def get_scores(self) -> Tuple[int, int]:
        """Returns (player1_score, player2_score)."""
        p1 = int(np.count_nonzero(self.board_status == -4))
        p2 = int(np.count_nonzero(self.board_status == 4))
        return p1, p2

    def available_moves(self) -> List[Tuple[str, List[int]]]:
        """Returns a list of all valid moves as (move_type, [row, col])."""
        moves = []
        for c in range(self.row_status.shape[0]):
            for r in range(self.row_status.shape[1]):
                if self.row_status[c, r] == 0:
                    moves.append(("row", [r, c]))

        for c in range(self.col_status.shape[0]):
            for r in range(self.col_status.shape[1]):
                if self.col_status[c, r] == 0:
                    moves.append(("col", [r, c]))

        return moves

    def apply_move(self, move_type: str, logical_position: List[int]) -> bool:
        """
        Applies a move to the game state.
        Returns True if a box was completed, False otherwise.
        """
        r = logical_position[0]
        c = logical_position[1]
        val = 1
        player_modifier = -1 if self.player1_turn else 1

        scored = False
        n_dots = self.get_dimensions()

        if c < (n_dots - 1) and r < (n_dots - 1):
            cur = abs(self.board_status[c, r]) + val
            self.board_status[c, r] = cur * player_modifier
            if abs(self.board_status[c, r]) == 4:
                scored = True

        if move_type == "row":
            self.row_status[c, r] = 1
            if c >= 1:
                cur = abs(self.board_status[c - 1, r]) + val
                self.board_status[c - 1, r] = cur * player_modifier
                if abs(self.board_status[c - 1, r]) == 4:
                    scored = True

        elif move_type == "col":
            self.col_status[c, r] = 1
            if r >= 1:
                cur = abs(self.board_status[c, r - 1]) + val
                self.board_status[c, r - 1] = cur * player_modifier
                if abs(self.board_status[c, r - 1]) == 4:
                    scored = True

        return scored

## MinimaxAgent Class

You are required to implement the core Minimax algorithm with **Alpha-Beta pruning**.

## Task 1: Implement the Evaluation Function

The evaluation function estimates how good a game state is for the AI agent. A good evaluation function considers:
- **Score Difference**: The primary factor (your score vs opponent's score)
- **Strategic Position**: Boxes that are almost complete, control of the board, etc.

You can create your own evaluation function or use a simple one based on score difference.

## Task 2: Implement the Minimax Algorithm with Alpha-Beta Pruning

The minimax algorithm explores the game tree to find the optimal move. **You MUST implement Alpha-Beta pruning** to make it efficient.

### Key Concepts to Implement:
1. **Minimax**: 
   - Maximizing player tries to maximize the score
   - Minimizing player tries to minimize the score
   - Recursively evaluate game states to find the best move

2. **Alpha-Beta Pruning** (REQUIRED):
   - Alpha (α): The best value the maximizer can guarantee
   - Beta (β): The best value the minimizer can guarantee
   - If β ≤ α at any point, prune (skip) the remaining branches
   - This dramatically reduces the number of nodes evaluated

3. **Depth Limiting**:
   - Use `max_depth` to limit how far ahead the algorithm looks
   - When depth limit is reached, use the evaluation function

In [136]:
class MinimaxAgent:
    def __init__(self, isPlayer1: bool = False):
        """
        Initialize the Minimax agent.
        Args:
            isPlayer1: True if this agent is Player 1, False if Player 2
        """
        self.isPlayer1 = isPlayer1
        self.nodes_evaluated = 0  # node Counter added for test

    @staticmethod
    def _evaluate(state: GameState, agentIsPlayer1: bool) -> float:
        """
        An evaluation function for non-terminal game states.
        This function should return a score indicating how favorable the state is
        for the agent. Higher scores mean better positions for the agent.
        """
        
        p1_score, p2_score = state.get_scores()
        
        if agentIsPlayer1:
            score_diff = p1_score - p2_score
            agent_score = p1_score
        else:
            score_diff = p2_score - p1_score
            agent_score = p2_score
        
        three_edge_boxes = np.count_nonzero(np.abs(state.board_status) == 3)
        two_edge_boxes = np.count_nonzero(np.abs(state.board_status) == 2)
        
        heuristic_value = (0.1 * two_edge_boxes) - (0.5 * three_edge_boxes)

        return score_diff + heuristic_value

    @staticmethod
    def _is_maximizing_turn(state: GameState, agentIsPlayer1: bool) -> bool:
        """
        TODO: Determine if the current turn is a maximizing turn for the agent.
        The agent maximizes when it's their turn, and minimizes when it's the opponent's turn.
        """
        return state.player1_turn == agentIsPlayer1

    @staticmethod
    def _order_moves(state: GameState, moves: List[Tuple[str, List[int]]]) -> List[Tuple[str, List[int]]]:
        """
        Helper method (PROVIDED): Orders moves for better alpha-beta pruning efficiency.
        Prioritizes moves that complete boxes, then safe moves, then risky moves.
        You can use this in your choose_move() and _minimax() methods.
        """
        completing_moves = []
        safe_moves = []
        risky_moves = []

        for move_type, pos in moves:
            test_state = state.clone()
            scored = test_state.apply_move(move_type, pos)

            if scored:
                completing_moves.append((move_type, pos))
            else:
                boxes_at_3 = int(np.count_nonzero(np.abs(test_state.board_status) == 3))
                if boxes_at_3 > int(np.count_nonzero(np.abs(state.board_status) == 3)):
                    risky_moves.append((move_type, pos))
                else:
                    safe_moves.append((move_type, pos))

        return completing_moves + safe_moves + risky_moves

    def choose_move(
        self, state: GameState, max_depth: Optional[int] = None, no_pruning: bool = False
    ) -> Tuple[Optional[str], Optional[List[int]], float]:
        """
        Choose the best move using the Minimax algorithm with Alpha-Beta pruning.
        This is the main entry point that starts the minimax search.
        Remember: You MUST implement alpha-beta pruning in this method!

        Hint: Use self._order_moves(state, moves) to get optimally ordered moves for better pruning.
        """

        best_move_type = None
        best_pos = None
        best_value = -math.inf 
        
        moves = state.available_moves()
        ordered_moves = self._order_moves(state, moves)

        alpha = -math.inf
        beta = math.inf
        
        for move_type, pos in ordered_moves:
            new_state = state.clone()
            scored = new_state.apply_move(move_type, pos)
            next_maximizing = True if scored else False
            value = self._minimax(new_state, 1, max_depth, alpha, beta, next_maximizing, no_pruning)
            
            if value > best_value:
                best_value = value
                best_move_type = move_type
                best_pos = pos
                

            alpha = max(alpha, value)
            
        return best_move_type, best_pos, best_value


    def _minimax(
        self,
        state: GameState,
        depth: int,
        max_depth: Optional[int],
        alpha: float,
        beta: float,
        maximizing: bool,
        no_pruning: bool = False # added no_pruning flag
    ) -> float:
        
        self.nodes_evaluated += 1
        
        moves = state.available_moves()
        ordered_moves = self._order_moves(state, moves)

        if maximizing:
            max_eval = -math.inf
            for move_type, pos in ordered_moves:
                new_state = state.clone()
                scored = new_state.apply_move(move_type, pos)
                next_maximizing = maximizing if scored else not maximizing

                eval_score = self._minimax(new_state, depth + 1, max_depth, alpha, beta, next_maximizing, no_pruning)
                max_eval = max(max_eval, eval_score)
                alpha = max(alpha, eval_score)
                
                if not no_pruning and beta <= alpha: 
                    break
            return max_eval
        
        else: # Minimizing
            min_eval = math.inf
            for move_type, pos in ordered_moves:
                new_state = state.clone()
                scored = new_state.apply_move(move_type, pos)
                next_maximizing = maximizing if scored else not maximizing
                
                eval_score = self._minimax(new_state, depth + 1, max_depth, alpha, beta, next_maximizing, no_pruning)
                min_eval = min(min_eval, eval_score)
                beta = min(beta, eval_score)

                if not no_pruning and beta <= alpha:
                    break
            return min_eval

## Testing Your Implementation

Once you've implemented the Minimax agent, you can test it by playing against it or watching it play against itself.

### Evaluation Steps

To test if your implementation is working correctly:

1. **Test with small depth limits** (e.g., 2-3) first to ensure correctness
2. **Verify alpha-beta pruning** is working by adding counters to track how many nodes are evaluated
3. **Compare performance**: Without pruning vs with pruning
4. **Test different evaluation functions** and see which performs better

In [ ]:
# TODO: Test your MinimaxAgent implementation here
# You can create simple test cases to verify your algorithm works correctly

# Example: Create a simple game state and test the agent
# number_of_dots = 3  # Start with a small board
# initial_state = make_initial_state(number_of_dots, player1_starts=False)

# agent = MinimaxAgent(isPlayer1=False)
# move_type, position, value = agent.choose_move(initial_state, max_depth=2)
# print(f"Best move: {move_type} at {position} with value {value}")


In [137]:
# Helper function for creating initial game states (useful for testing)
def make_initial_state(number_of_dots: int, player1_starts: bool = True) -> GameState:
    """
    Creates a new game state with an empty board.
    """
    board_status = np.zeros((number_of_dots - 1, number_of_dots - 1), dtype=int)
    row_status = np.zeros((number_of_dots, number_of_dots - 1), dtype=int)
    col_status = np.zeros((number_of_dots - 1, number_of_dots), dtype=int)
    return GameState(
        board_status=board_status, 
        row_status=row_status, 
        col_status=col_status, 
        player1_turn=player1_starts
    )

In [138]:
## TEST CASE 1: Basic Correctness (Depth 2)
print("--- Test 1: Basic Correctness (Depth 2) ---")
number_of_dots = 3  # 2x2 board
initial_state = make_initial_state(number_of_dots, player1_starts=False) # Agent (P2) starts

agent = MinimaxAgent(isPlayer1=False)
agent.nodes_evaluated = 0 # Reset counter before search

move_type, position, value = agent.choose_move(initial_state, max_depth=2)

print(f"Board size: {number_of_dots-1}x{number_of_dots-1}")
print(f"Max Depth: 2")
print(f"Best move: {move_type} at {position}")
print(f"Minimax Value: {value:.2f}")
print(f"Nodes Evaluated: {agent.nodes_evaluated}\n")

# Expected Outcome: The agent should choose a move that doesn't immediately set up P1 to score.

--- Test 1: Basic Correctness (Depth 2) ---
Board size: 2x2
Max Depth: 2
Best move: None at None
Minimax Value: -inf
Nodes Evaluated: 208326



In [143]:
## TEST CASE 2 & 3: Performance Comparison (Depth 3)
print("--- Test 2 & 3: Performance Comparison (Depth 3) ---")
number_of_dots = 3 # 2x2 board
initial_state = make_initial_state(number_of_dots, player1_starts=True) # P1 (Human) starts

# With Alpha-Beta Pruning
agent_pruned = MinimaxAgent(isPlayer1=True)
agent_pruned.nodes_evaluated = 0
move_type_p, position_p, value_p = agent_pruned.choose_move(initial_state, max_depth=4)

print("A. Minimax WITH Alpha-Beta Pruning:")
print(f"  Best move: {move_type_p} at {position_p}")
print(f"  Nodes Evaluated: {agent_pruned.nodes_evaluated}")

# Without Alpha-Beta Pruning
agent_unpruned = MinimaxAgent(isPlayer1=True)
agent_unpruned.nodes_evaluated = 0
move_type_up, position_up, value_up = agent_unpruned.choose_move(initial_state, max_depth=4, no_pruning=True)

print("\nB. Minimax WITHOUT Alpha-Beta Pruning (Simulated):")
print(f"  Best move: {move_type_up} at {position_up}")
print(f"  Nodes Evaluated: {agent_unpruned.nodes_evaluated}")

# Verification: The *nodes evaluated* in A should be significantly lower than in B.

--- Test 2 & 3: Performance Comparison (Depth 3) ---
A. Minimax WITH Alpha-Beta Pruning:
  Best move: None at None
  Nodes Evaluated: 208326


KeyboardInterrupt: 

In [144]:
## TEST CASE 4: Evaluation Function Check
print("\n--- Test 4: Evaluation Function Check ---")

state_4 = make_initial_state(3, player1_starts=True) 

state_4.col_status[0, 0] = 1
state_4.row_status[0, 0] = 1
state_4.col_status[0, 1] = 1
state_4.board_status[0, 0] = -3

risky_move = ("row", [0, 1]) 
safe_move = ("row", [0, 0]) 

agent_eval = MinimaxAgent(isPlayer1=True)
agent_eval.nodes_evaluated = 0
move_type_e, position_e, value_e = agent_eval.choose_move(state_4, max_depth=1)

print(f"Current P1 Score: {state_4.get_scores()[0]}")
print(f"Available moves (partial list): {state_4.available_moves()[:2]}")
print(f"Agent's optimal move (Depth 1): {move_type_e} at {position_e} (Value: {value_e:.2f})")

# If the agent chooses the scoring move, the value will be high (1 point)
# If the agent chooses the safe move, the value will be lower (0 points, but good position)


--- Test 4: Evaluation Function Check ---
Current P1 Score: 0
Available moves (partial list): [('row', [1, 0]), ('row', [0, 1])]
Agent's optimal move (Depth 1): None at None (Value: -inf)


---

## Dots and Boxes Game Implementation

Below is the complete implementation of the Dots and Boxes game with a graphical user interface (GUI). This code is **provided for you** and integrates your MinimaxAgent to play against you or itself.

### How to Play:

1. **Run all the cells** in this section to start the game
2. The game will open in a new window
3. Click on the grid lines to draw edges
4. Try to complete boxes to score points
5. The AI (Player 2) will automatically make moves using your Minimax implementation
6. The game ends when all boxes are completed

### Game Configuration:

You can adjust these parameters:
- `number_of_dots`: Grid size (default is 6x6)
- `MAX_AI_DEPTH`: How many moves ahead the AI looks (default is 2)
- `ai_mode`: Set to `True` to play against AI, `False` for 2-player mode

In [121]:
# Game Configuration
MAX_AI_DEPTH = 2

size_of_board = 600
number_of_dots = 6
symbol_size = (size_of_board / 3 - size_of_board / 8) / 2
symbol_thickness = 50
dot_color = "#FFFFFF"
player1_color = '#0492CF'
player1_color_light = '#67B0CF'
player2_color = '#EE4035'
player2_color_light = '#EE7E77'
text_color = "#FFFFFF"
dot_width = 0.25*size_of_board/number_of_dots
edge_width = 0.1*size_of_board/number_of_dots
distance_between_dots = size_of_board / (number_of_dots)

In [122]:
class Dots_and_Boxes():
    def __init__(self):
        self.window = Tk()
        self.window.title('Dots_and_Boxes')
        self.canvas = Canvas(self.window, width=size_of_board, height=size_of_board)
        self.canvas.pack()
        self.window.bind('<Button-1>', self.click)
        self.player1_starts = True
        self.ai_agent = MinimaxAgent(isPlayer1=False)
        self.ai_mode = True
        self.ai_max_depth = MAX_AI_DEPTH
        self.refresh_board()
        self.play_again()

    def play_again(self):
        self.refresh_board()
        self.board_status = np.zeros(shape=(number_of_dots - 1, number_of_dots - 1))
        self.row_status = np.zeros(shape=(number_of_dots, number_of_dots - 1))
        self.col_status = np.zeros(shape=(number_of_dots - 1, number_of_dots))
        self.pointsScored = False
        self.player1_starts = not self.player1_starts
        self.player1_turn = self.player1_starts
        self.reset_board = False
        self.turntext_handle = []
        self.already_marked_boxes = []
        self.ai_agent.isPlayer1 = False
        self.display_turn_text()
        self.ai_move_if_needed()

    def mainloop(self):
        self.window.mainloop()

    def is_grid_occupied(self, logical_position, type):
        r = logical_position[0]
        c = logical_position[1]
        occupied = True
        if type == 'row' and self.row_status[c][r] == 0:
            occupied = False
        if type == 'col' and self.col_status[c][r] == 0:
            occupied = False
        return occupied

    def convert_grid_to_logical_position(self, grid_position):
        grid_position = np.array(grid_position)
        position = (grid_position-distance_between_dots/4)//(distance_between_dots/2)

        type = False
        logical_position = []
        if position[1] % 2 == 0 and (position[0] - 1) % 2 == 0:
            r = int((position[0]-1)//2)
            c = int(position[1]//2)
            logical_position = [r, c]
            type = 'row'
        elif position[0] % 2 == 0 and (position[1] - 1) % 2 == 0:
            c = int((position[1] - 1) // 2)
            r = int(position[0] // 2)
            logical_position = [r, c]
            type = 'col'

        return logical_position, type

    def pointScored(self):
        self.pointsScored = True

    def mark_box(self):
        boxes = np.argwhere(self.board_status == -4)
        for box in boxes:
            if tuple(box) not in [tuple(b) for b in self.already_marked_boxes]:
                self.already_marked_boxes.append(list(box))
                color = player1_color_light
                self.shade_box(box, color)

        boxes = np.argwhere(self.board_status == 4)
        for box in boxes:
            if tuple(box) not in [tuple(b) for b in self.already_marked_boxes]:
                self.already_marked_boxes.append(list(box))
                color = player2_color_light
                self.shade_box(box, color)

    def get_game_state_from_gui(self):
        """Helper method to convert GUI state to GameState object."""
        return GameState(
            board_status=self.board_status.copy(),
            row_status=self.row_status.copy(),
            col_status=self.col_status.copy(),
            player1_turn=self.player1_turn
        )

    def is_ai_turn(self):
        return (not self.ai_agent.isPlayer1 and not self.player1_turn) or \
               (self.ai_agent.isPlayer1 and self.player1_turn)

    def ai_move_if_needed(self):
        if self.reset_board or not self.ai_mode or not self.is_ai_turn():
            return

        while True:
            if self.is_gameover():
                break

            state = GameState(
                board_status=self.board_status.copy(),
                row_status=self.row_status.copy(),
                col_status=self.col_status.copy(),
                player1_turn=self.player1_turn
            )

            move_type, pos, _ = self.ai_agent.choose_move(state, max_depth=self.ai_max_depth)
            if move_type is None:
                break

            self.update_board(move_type, pos)
            self.make_edge(move_type, pos)
            self.mark_box()
            self.refresh_board()

            if not self.pointsScored:
                self.player1_turn = not self.player1_turn

            self.pointsScored = False

            if self.is_gameover():
                self.display_gameover()
                break

            if not self.is_ai_turn():
                self.display_turn_text()
                break

    def update_board(self, type, logical_position):
        r = logical_position[0]
        c = logical_position[1]
        val = 1
        playerModifier = -1 if self.player1_turn else 1

        if c < (number_of_dots-1) and r < (number_of_dots-1):
            self.board_status[c][r] = (abs(self.board_status[c][r]) + val) * playerModifier
            if abs(self.board_status[c][r]) == 4:
                self.pointScored()

        if type == 'row':
            self.row_status[c][r] = 1
            if c >= 1:
                self.board_status[c-1][r] = (abs(self.board_status[c-1][r]) + val) * playerModifier
                if abs(self.board_status[c-1][r]) == 4:
                    self.pointScored()

        elif type == 'col':
            self.col_status[c][r] = 1
            if r >= 1:
                self.board_status[c][r-1] = (abs(self.board_status[c][r-1]) + val) * playerModifier
                if abs(self.board_status[c][r-1]) == 4:
                    self.pointScored()

    def is_gameover(self):
        return (self.row_status == 1).all() and (self.col_status == 1).all()

    def make_edge(self, type, logical_position):
        if type == 'row':
            start_x = distance_between_dots/2 + logical_position[0]*distance_between_dots
            end_x = start_x+distance_between_dots
            start_y = distance_between_dots/2 + logical_position[1]*distance_between_dots
            end_y = start_y
        elif type == 'col':
            start_y = distance_between_dots / 2 + logical_position[1] * distance_between_dots
            end_y = start_y + distance_between_dots
            start_x = distance_between_dots / 2 + logical_position[0] * distance_between_dots
            end_x = start_x

        if self.player1_turn:
            color = player1_color
        else:
            color = player2_color
        self.canvas.create_line(start_x, start_y, end_x, end_y, fill=color, width=edge_width)

    def display_gameover(self):
        player1_score = len(np.argwhere(self.board_status == -4))
        player2_score = len(np.argwhere(self.board_status == 4))

        if player1_score > player2_score:
            text = 'Winner: Player 1 '
            color = player1_color
        elif player2_score > player1_score:
            text = 'Winner: Player 2 '
            color = player2_color
        else:
            text = 'Its a tie'
            color = 'gray'

        self.canvas.delete("all")
        self.canvas.create_text(size_of_board / 2, size_of_board / 3, font="cmr 60 bold", fill=color, text=text)

        score_text = 'Scores \n'
        self.canvas.create_text(size_of_board / 2, 5 * size_of_board / 8, font="cmr 40 bold", fill=text_color,
                                text=score_text)

        score_text = 'Player 1 : ' + str(player1_score) + '\n'
        score_text += 'Player 2 : ' + str(player2_score) + '\n'
        self.canvas.create_text(size_of_board / 2, 3 * size_of_board / 4, font="cmr 30 bold", fill=text_color,
                                text=score_text)
        self.reset_board = True

        score_text = 'Click to play again \n'
        self.canvas.create_text(size_of_board / 2, 15 * size_of_board / 16, font="cmr 20 bold", fill="gray",
                                text=score_text)

    def refresh_board(self):
        for i in range(number_of_dots):
            x = i*distance_between_dots+distance_between_dots/2
            self.canvas.create_line(x, distance_between_dots/2, x,
                                    size_of_board-distance_between_dots/2,
                                    fill='gray', dash = (2, 2))
            self.canvas.create_line(distance_between_dots/2, x,
                                    size_of_board-distance_between_dots/2, x,
                                    fill='gray', dash=(2, 2))

        for i in range(number_of_dots):
            for j in range(number_of_dots):
                start_x = i*distance_between_dots+distance_between_dots/2
                end_x = j*distance_between_dots+distance_between_dots/2
                self.canvas.create_oval(start_x-dot_width/2, end_x-dot_width/2, start_x+dot_width/2,
                                        end_x+dot_width/2, fill=dot_color,
                                        outline=dot_color)

    def display_turn_text(self):
        text = 'Next turn: '
        if self.player1_turn:
            text += 'Player1'
            color = player1_color
        else:
            text += 'Player2'
            color = player2_color

        self.canvas.delete(self.turntext_handle)
        self.turntext_handle = self.canvas.create_text(size_of_board - 5*len(text),
                                                       size_of_board-distance_between_dots/8,
                                                       font="cmr 15 bold", text=text, fill=color)

    def shade_box(self, box, color):
        start_x = distance_between_dots / 2 + box[1] * distance_between_dots + edge_width/2
        start_y = distance_between_dots / 2 + box[0] * distance_between_dots + edge_width/2
        end_x = start_x + distance_between_dots - edge_width
        end_y = start_y + distance_between_dots - edge_width
        self.canvas.create_rectangle(start_x, start_y, end_x, end_y, fill=color, outline='')

    def click(self, event):
        if not self.reset_board:
            grid_position = [event.x, event.y]
            logical_positon, valid_input = self.convert_grid_to_logical_position(grid_position)
            if valid_input and not self.is_grid_occupied(logical_positon, valid_input):
                self.update_board(valid_input, logical_positon)
                self.make_edge(valid_input, logical_positon)
                self.mark_box()
                self.refresh_board()

                if not self.pointsScored:
                    self.player1_turn = not self.player1_turn

                self.pointsScored = False

                if self.is_gameover():
                    self.display_gameover()
                else:
                    self.display_turn_text()
                    if self.ai_mode:
                        self.ai_move_if_needed()
        else:
            self.canvas.delete("all")
            self.play_again()
            self.reset_board = False

## Run the Game

Execute the cell below to start playing! Make sure you have implemented the MinimaxAgent methods before running this.

In [124]:
# Start the game
game_instance = Dots_and_Boxes()
game_instance.mainloop()

## **REPORT* * 
### Answer to the questions of part 2 :

##### *question 1*
```
Search Depth (d)    Execution Time      Nodes Evaluated     Decision Accuracy (Win Rate)
Low (e.g., d=2)     Low (≈O(b2))        Low                 Poor/Moderate. The AI only sees immediate gains/losses. Prone to blunders that lose the game in the long run.
Medium (e.g., d=4)  Moderate (≈O(b4))   Moderate            Good. Balances foresight with speed. Can usually avoid traps and set up multi-move sequences.
High (e.g., d=6)    High/Very High  (≈O(b6)) High/Very High  Excellent. Sees deep into complex scenarios (especially mid-game). Computationally expensive and slow.
```

Trade-off Between Accuracy and EfficiencyIncreasing the search depth creates a direct trade-off:Decision Accuracy (Foresight): Higher depth increases the ability to see further ahead, which is critical in a game like Dots and Boxes where a single move can lead to a long chain of points (a "chain play"). Deeper search leads to higher accuracy and a better win rate. * Computational Efficiency (Speed): The required time and number of nodes evaluated grow exponentially with depth, O(b^d).
This exponential growth severely limits how deep the search can go before moves take too long, making the agent impractical or unusable.Suggesting an Optimal DepthThe optimal depth balances the desire for strong play with the need for timely moves. 
Initial/Endgame: A lower depth (d=2 or d=4) might be sufficient at the start when the board is open.
Mid-Game/Complexity: For a small board, d=6 might be feasible. For a standard 5 \times 5 board, d=4 is often the maximum practical depth.A good optimized depth for a competitive game is often found empirically: Find the maximum depth d where the average move time is acceptable. Based on typical Dots and Boxes complexity, this is often d=4 or d=5 when Alpha-Beta pruning is fully functional.

##### *question 2*
The order of moves is highly critical to the performance of Alpha-Beta pruning:Efficiency: Alpha-Beta Pruning works by checking if the current branch's guaranteed value ($\alpha$ for the maximizer, $\beta$ for the minimizer) makes exploring the rest of the branch redundant. To maximize pruning, the algorithm must evaluate the best moves first.Optimal Ordering: If the best move is checked early, the $\alpha$ value rises rapidly and the $\beta$ value drops rapidly, quickly closing the $[\alpha, \beta]$ window and allowing early pruning of suboptimal branches.Worst-Case Scenario: If moves are ordered poorly (e.g., randomly), the algorithm might evaluate a much larger portion of the tree, approaching the time complexity of the unpruned Minimax search (O(b^d)).Analysis of the Provided _order_moves StrategyThe provided _order_moves strategy is a strong heuristic for Dots and Boxes:Prioritize Completing Moves: Moves that scored=True are placed first. These are the most critical moves as they give the current player an extra turn, which is hugely beneficial and must be explored early.Prioritize Safe Moves: Moves that don't increase the count of 3-edge boxes are considered "safe." These are placed second. This avoids immediately setting up the opponent for a score.Prioritize Risky Moves: Moves that create a new 3-edge box (boxes_at_3 count increases) are deemed "risky" and placed last. These moves are least likely to be the best, making them ideal candidates for pruning.Conclusion: This ordering is specifically tailored to the game mechanics and is effective for maximizing the efficiency of Alpha-Beta pruning by prioritizing high-value (scoring) and low-risk (safe) moves.Suggested Strategic EnhancementA strategic enhancement to the evaluation function or move ordering would involve recognizing and prioritizing Double-Cross threats or Chain-Play potential.Suggested Enhancement: Analyze Chain-Play PotentialThe current heuristic only penalizes the state where a box is at 3 edges. A better strategy is to detect moves that start a guaranteed sequence of scores (a chain play).Implement a Lookahead Check in _order_moves: For all moves that result in a score (completing_moves), perform a very shallow (depth 1 or 2) search to see how many additional consecutive scoring moves follow.Refined Ordering: Rank the completing_moves list by the length of the scoring chain they initiate. A move that scores 3 boxes consecutively is vastly better than a move that scores 1 box and then forces a handover.